# [BioRAG](https://zenodo.org/records/13119834) e [GeneRAG](https://sites.google.com/view/gene-rag/home?authuser=0) : incorporando planilhas

In [ ]:
!pip install requests bs4 html2text mygene
!pip install -qU langchain-openai
!pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4
!wget

wget: missing URL
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.


In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# arquivo no drive
file_name = '/content/drive/MyDrive/Homo_sapiens.gene_info.csv'

df = pd.read_csv(file_name)
df.head()


Mounted at /content/drive


,Unnamed: 0,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,0,9606,1,A1BG,-,A1B|ABG|GAB|HYST2477,MIM:138670|HGNC:HGNC:5|Ensembl:ENSG00000121410,19,19q13.43,alpha-1-B glycoprotein,protein-coding,A1BG,alpha-1-B glycoprotein,O,alpha-1B-glycoprotein|HEL-S-163pA|epididymis s...,20200313,-
1,1,9606,2,A2M,-,A2MD|CPAMD5|FWP007|S863-7,MIM:103950|HGNC:HGNC:7|Ensembl:ENSG00000175899,12,12p13.31,alpha-2-macroglobulin,protein-coding,A2M,alpha-2-macroglobulin,O,alpha-2-macroglobulin|C3 and PZP-like alpha-2-...,20200405,-
2,2,9606,3,A2MP1,-,A2MP,HGNC:HGNC:8|Ensembl:ENSG00000256069,12,12p13.31,alpha-2-macroglobulin pseudogene 1,pseudo,A2MP1,alpha-2-macroglobulin pseudogene 1,O,pregnancy-zone protein pseudogene,20200313,-
3,3,9606,9,NAT1,-,AAC1|MNAT|NAT-1|NATI,MIM:108345|HGNC:HGNC:7645|Ensembl:ENSG00000171428,8,8p22,N-acetyltransferase 1,protein-coding,NAT1,N-acetyltransferase 1,O,arylamine N-acetyltransferase 1|N-acetyltransf...,20200313,-
4,4,9606,10,NAT2,-,AAC2|NAT-2|PNAT,MIM:612182|HGNC:HGNC:7646|Ensembl:ENSG00000156006,8,8p22,N-acetyltransferase 2,protein-coding,NAT2,N-acetyltransferase 2,O,arylamine N-acetyltransferase 2|N-acetyltransf...,20200405,-


In [ ]:
df.columns

Index(['Unnamed: 0', '#tax_id', 'GeneID', 'Symbol', 'LocusTag', 'Synonyms',
       'dbXrefs', 'chromosome', 'map_location', 'description', 'type_of_gene',
       'Symbol_from_nomenclature_authority',
       'Full_name_from_nomenclature_authority', 'Nomenclature_status',
       'Other_designations', 'Modification_date', 'Feature_type'],
      dtype='object')

In [ ]:
import requests
from bs4 import BeautifulSoup
import html2text
import pandas as pd
import json
import pickle
from concurrent.futures import ThreadPoolExecutor


gene_info_table = df
# use top 10 gene names for testing and convert them into dict
top_k = 100
gene_object_list = gene_info_table.to_dict(orient='records')[:top_k]

parts_to_remove = [
    "##  Summary\n",
    "NEW",
    'Try the newGene table',
    'Try the newTranscript table',
    '**',
    "\nGo to the top of the page Help\n"
]


def rough_text_from_gene_name(gene_number):
    # get url
    url = f"https://www.ncbi.nlm.nih.gov/gene/{gene_number}"
    # Send a GET request to the URL
    summary_text = ''
    soup = None
    try:
        response = requests.get(url, timeout=30)
    except requests.exceptions.Timeout:
        print('time out')
        return ((summary_text, soup))
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the "summary" tab content by inspecting the page's structure
        summary_tab = soup.find('div', {'class': 'rprt-section gene-summary'})

        # Check if the "summary" tab content is found
        if summary_tab:
            # Convert the HTML to plain text using html2text
            html_to_text = html2text.HTML2Text()
            html_to_text.ignore_links = True  # Ignore hyperlinks

            # Extract the plain text from the "summary" tab
            summary_text = html_to_text.handle(str(summary_tab))
            # Remove the specified parts from the original text
            for part in parts_to_remove:
                summary_text = summary_text.replace(part, ' ')
                # Replace '\n' with a space
            summary_text = summary_text.replace('\n', ' ')

            # Reduce multiple spaces into one space
            summary_text = ' '.join(summary_text.split())
            # Print or save the extracted text
        else:
            print("Summary tab not found on the page.")
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
    return ((summary_text, soup))


with ThreadPoolExecutor(max_workers=15) as executor:
    futures = []
    for gene in gene_object_list:
        gene_number = gene['GeneID']
        futures.append(executor.submit(rough_text_from_gene_name, gene_number))

    for gene, future in zip(gene_object_list, futures):
        gene['summary_text'], _ = future.result()

# Save the extracted data to a CSV file
output_csv_path = "./gene_summary_text.csv"
gene_summary_text_table = pd.DataFrame(gene_object_list)
gene_summary_text_table.to_csv(output_csv_path, index=False)

In [ ]:
df1 = pd.read_csv("./gene_summary_text.csv")
df1.head()

,Unnamed: 0,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type,summary_text
0,0,9606,1,A1BG,-,A1B|ABG|GAB|HYST2477,MIM:138670|HGNC:HGNC:5|Ensembl:ENSG00000121410,19,19q13.43,alpha-1-B glycoprotein,protein-coding,A1BG,alpha-1-B glycoprotein,O,alpha-1B-glycoprotein|HEL-S-163pA|epididymis s...,20200313,-,Official Symbol A1BGprovided by HGNC Official ...
1,1,9606,2,A2M,-,A2MD|CPAMD5|FWP007|S863-7,MIM:103950|HGNC:HGNC:7|Ensembl:ENSG00000175899,12,12p13.31,alpha-2-macroglobulin,protein-coding,A2M,alpha-2-macroglobulin,O,alpha-2-macroglobulin|C3 and PZP-like alpha-2-...,20200405,-,Official Symbol A2Mprovided by HGNC Official F...
2,2,9606,3,A2MP1,-,A2MP,HGNC:HGNC:8|Ensembl:ENSG00000256069,12,12p13.31,alpha-2-macroglobulin pseudogene 1,pseudo,A2MP1,alpha-2-macroglobulin pseudogene 1,O,pregnancy-zone protein pseudogene,20200313,-,Official Symbol A2MP1provided by HGNC Official...
3,3,9606,9,NAT1,-,AAC1|MNAT|NAT-1|NATI,MIM:108345|HGNC:HGNC:7645|Ensembl:ENSG00000171428,8,8p22,N-acetyltransferase 1,protein-coding,NAT1,N-acetyltransferase 1,O,arylamine N-acetyltransferase 1|N-acetyltransf...,20200313,-,Official Symbol NAT1provided by HGNC Official ...
4,4,9606,10,NAT2,-,AAC2|NAT-2|PNAT,MIM:612182|HGNC:HGNC:7646|Ensembl:ENSG00000156006,8,8p22,N-acetyltransferase 2,protein-coding,NAT2,N-acetyltransferase 2,O,arylamine N-acetyltransferase 2|N-acetyltransf...,20200405,-,Official Symbol NAT2provided by HGNC Official ...


In [ ]:
df1.columns

Index(['Unnamed: 0', '#tax_id', 'GeneID', 'Symbol', 'LocusTag', 'Synonyms',
       'dbXrefs', 'chromosome', 'map_location', 'description', 'type_of_gene',
       'Symbol_from_nomenclature_authority',
       'Full_name_from_nomenclature_authority', 'Nomenclature_status',
       'Other_designations', 'Modification_date', 'Feature_type',
       'summary_text'],
      dtype='object')

In [ ]:
from google.colab import userdata


In [ ]:
# @title

import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


In [ ]:
# @title
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")


In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import CSVLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")
# Load, chunk and index the contents of the blog.
loader = CSVLoader(
    file_path="./gene_summary_text.csv",
    #file_path="/content/drive/MyDrive/Homo_sapiens.gene_info.csv",
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [ ]:
rag_chain.invoke("Where is the map location of SERPINA3?")

'The map location of SERPINA3 is 14q32.13 on chromosome 14.'

In [ ]:
rag_chain.invoke("could you list top6 tissues where CFTR expresses in highest level?")

'The top six tissues where CFTR expresses at the highest levels include the urinary bladder, thyroid, spleen, gall bladder, lung, and appendix. Specific RPKM values indicate their expression: urinary bladder (30.9), thyroid (26.8), spleen (22.5), gall bladder (5.8), lung (8.5), and appendix (19.0). These values highlight the varying levels of CFTR expression across different tissues.'

In [ ]:
rag_chain.invoke("What is the full name of A1BG and what is it?")

'The full name of A1BG is alpha-1-B glycoprotein. It is a protein-coding gene in humans that encodes a plasma glycoprotein of unknown function, showing sequence similarity to some immunoglobulin supergene family member proteins.'

# Opção com [LangChain](https://python.langchain.com/v0.1/docs/langsmith/walkthrough/) (ainda experimental)

In [ ]:
!pip install langchain-experimental
!pip install langchain-openai

In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI, OpenAI

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# nome do arquivo no drive
file_name = '/content/drive/MyDrive/Homo_sapiens.gene_info.csv'

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(file_name)
df.head()

In [ ]:

agent = create_csv_agent(
    ChatOpenAI(temperature=0, model="gpt-4o-mini"),
    file_name,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True
)

In [ ]:
agent.run("What is the full name of A1BG and what is it?")

In [ ]:
agent.run("Where is the map location of SERPINA3?")

In [ ]:
agent.run("could you list top6 tissues where CFTR expresses in highest level?")